In [5]:
#Master Working Backup
import os
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from imagekitio import ImageKit
from openai import OpenAI
from dotenv import load_dotenv
from PIL import Image

ticker = "SPY"

load_dotenv()

client = OpenAI()
screenshot_dir = r"F:\StillWater\Apps\Pecuniary\smartinvestor\tools\FalconEye\screenshots"
os.makedirs(screenshot_dir, exist_ok=True)

# ImageKit Credentials
imagekit = ImageKit(
    private_key=os.getenv("IMAGEKIT_PRIVATE_KEY"),
    public_key=os.getenv("IMAGEKIT_PUBLIC_KEY"),
    url_endpoint=os.getenv("IMAGEKIT_URL_ENDPOINT")
)

# Function to Capture TradingView Chart
def capture_tradingview_chart(symbol):
    """Captures and converts a TradingView chart screenshot to a proper PNG."""
    try:
        print("Initializing WebDriver...")
        chrome_options = Options()
        chrome_options.add_argument("--window-size=1920x1080")
        chrome_options.add_argument("--disable-blink-features=AutomationControlled")
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")

        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)

        tradingview_url = f"https://www.tradingview.com/chart/?symbol={symbol}"
        print(f"Opening TradingView URL: {tradingview_url}")
        driver.get(tradingview_url)
        time.sleep(5)

        raw_screenshot_path = os.path.join(screenshot_dir, f"{symbol}_chart_raw.png")
        proper_screenshot_path = os.path.join(screenshot_dir, f"{symbol}_chart.png")

        # Take a screenshot (raw format)
        print("Taking screenshot...")
        driver.save_screenshot(raw_screenshot_path)
        driver.quit()

        # Convert screenshot to proper PNG format
        print("Converting screenshot to a valid PNG format...")
        img = Image.open(raw_screenshot_path)
        img = img.convert("RGB")  # Ensures proper color format
        img.save(proper_screenshot_path, "PNG")

        # Remove the raw screenshot
        os.remove(raw_screenshot_path)

        if os.path.exists(proper_screenshot_path):
            print(f"Chart successfully saved: {proper_screenshot_path}")
            return proper_screenshot_path
        else:
            print("Screenshot failed, file not found!")
            return None

    except Exception as e:
        print(f"Error capturing TradingView chart: {e}")
        return None

# Function to Upload Image to ImageKit
def upload_image_to_imagekit(image_path):
    """Uploads an image to ImageKit and returns the HTTPS URL."""
    try:
        if not os.path.exists(image_path):
            return "Error: Image file not found."

        print(f"Uploading {image_path} to ImageKit...")

        with open(image_path, "rb") as img_file:
            upload = imagekit.upload(
                file=img_file,
                file_name=os.path.basename(image_path)
            )

        print("Upload response:", upload.response_metadata.raw)
        return upload.url
    except Exception as e:
        print(f"Error uploading image: {e}")
        return None

# AI Analysis Function
def generate_ai_analysis_from_image(image_url):
    """Uses OpenAI's vision model to analyze a TradingView chart image via HTTPS URL."""
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a stock market expert analyzing candlestick charts."},
                {"role": "user", "content": [
                    {"type": "text", "text": "Analyze the stock chart in this image and provide insights on the perceived directional momentum sentiment, trends, volatility, and possible price action in a neat table report. Limit your response to 450 tokens and indicate the perceived sentiment."},
                    {"type": "image_url", "image_url": {"url": image_url}}
                ]}
            ]
        )

        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error generating analysis: {e}"

# **Full Execution Function**
def analyze_stock_chart(ticker):
    """Runs the complete process: Capture TradingView chart, upload image, and get AI analysis."""
    print("\n=== Capturing TradingView Chart ===")
    chart_path = capture_tradingview_chart(ticker)

    if not chart_path:
        print("Error: TradingView chart capture failed.")
        return

    print("\nUploading image to ImageKit...")
    image_url = upload_image_to_imagekit(chart_path)

    if not image_url or "Error" in image_url:
        print(f"Image upload failed: {image_url}")
        return

    print(f"Image uploaded successfully: {image_url}")

    print("\nRunning AI analysis on the uploaded chart...")
    ai_report = generate_ai_analysis_from_image(image_url)

    if os.path.exists(chart_path):
        os.remove(chart_path)
        print(f"Deleted local image: {chart_path}")

    print("\n=== AI-Generated TradingView Chart Analysis ===\n")
    print(ai_report)

    #return ai_report

# **Run for a sample stock**
analyze_stock_chart(ticker)


=== Capturing TradingView Chart ===
Initializing WebDriver...
Opening TradingView URL: https://www.tradingview.com/chart/?symbol=SPY
Taking screenshot...
Converting screenshot to a valid PNG format...
Chart successfully saved: F:\StillWater\Apps\Pecuniary\smartinvestor\tools\FalconEye\screenshots\SPY_chart.png

Uploading image to ImageKit...
Uploading F:\StillWater\Apps\Pecuniary\smartinvestor\tools\FalconEye\screenshots\SPY_chart.png to ImageKit...
Upload response: {'fileId': '68940e895c7cd75eb833fcb2', 'name': 'SPY_chart_um5g2zLRD.png', 'size': 80199, 'versionInfo': {'id': '68940e895c7cd75eb833fcb2', 'name': 'Version 1'}, 'filePath': '/SPY_chart_um5g2zLRD.png', 'url': 'https://ik.imagekit.io/fwe3vbq1p/SPY_chart_um5g2zLRD.png', 'fileType': 'image', 'height': 824, 'width': 1295, 'thumbnailUrl': 'https://ik.imagekit.io/fwe3vbq1p/tr:n-ik_ml_thumbnail/SPY_chart_um5g2zLRD.png', 'AITags': None, 'description': None}
Image uploaded successfully: https://ik.imagekit.io/fwe3vbq1p/SPY_chart_um5

In [ ]:
pip install openai

In [ ]:
#Enhanced Tinimport os
import os
import time
import json
from dotenv import load_dotenv
from openai import OpenAI
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from PIL import Image
import tinytroupe
from tinytroupe.agent import TinyPerson, FilesAndWebGroundingFaculty
from tinytroupe.environment import TinyWorld

# Load environment variables
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)
screenshot_dir = r"F:\StillWater\Apps\Pecuniary\smartinvestor\tools\FalconEye\screenshots"
os.makedirs(screenshot_dir, exist_ok=True)

memory_file = "agent_memory.json"

# Initialize agents with grounding faculty
grounding_faculty = FilesAndWebGroundingFaculty()

# Initialize agents
leslie = TinyPerson("Leslie")
leslie.define("age", 75)
leslie.define("occupation", "Baker, Mechanic, Accountant")
leslie.define("personality", {"traits": [
    "Patient and analytical.",
    "Enjoys explaining solutions to problems with numbers.",
    "Friendly and a good active listener."
]})
leslie.add_mental_faculties([grounding_faculty])

ashley = TinyPerson("Ashley")
ashley.define("age", 21)
ashley.define("occupation", "Model, Chemist, Gymnast")
ashley.define("personality", {"traits": [
    "Curious and analytical.",
    "Loves cooking healthy food.",
    "Very playful and energetic."
]})
ashley.add_mental_faculties([grounding_faculty])

# Create world
environment = TinyWorld("Trading Room", [leslie, ashley])
environment.make_everyone_accessible()

# Function to query OpenAI's GPT-4o
def get_gpt_response(prompt):
    try:
        completions = client.chat.completions.create(
            model="gpt-4",  # Changed from gpt-4o to gpt-4
            messages=[{"role": "system", "content": "You are an expert stock analyst guiding two AI personas."},
                    {"role": "user", "content": prompt}]
        )
        return completions.choices[0].message.content
    except Exception as e:
        print(f"Error getting GPT response: {e}")
        return f"Error: {str(e)}"

# Function to capture TradingView chart
def capture_tradingview_chart(symbol):
    try:
        options = Options()
        options.add_argument("--window-size=1920x1080")
        options.add_argument("--headless")  # Run in headless mode
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(f"https://www.tradingview.com/chart/?symbol={symbol}")
        time.sleep(5)
        screenshot_path = os.path.join(screenshot_dir, f"{symbol}_chart.png")
        driver.save_screenshot(screenshot_path)
        driver.quit()
        return screenshot_path
    except Exception as e:
        print(f"Error capturing chart: {e}")
        return None

# Function to analyze chart using GPT-4 Vision
def analyze_chart_with_gpt4o(image_path):
    try:
        with open(image_path, "rb") as image_file:
            # Encode the image as base64
            import base64
            image_base64 = base64.b64encode(image_file.read()).decode('utf-8')
            
        response = client.chat.completions.create(
            model="gpt-4.1",  # Using GPT-4 Vision model
            messages=[
                {
                    "role": "system",
                    "content": "Analyze the stock chart image and provide insights."
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": "Describe trends, volatility, and possible price action."
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/png;base64,{image_base64}"
                            }
                        }
                    ]
                }
            ],
            max_tokens=500
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error analyzing chart: {e}")
        return f"Error analyzing chart: {str(e)}"

# Function to save agent memory
def save_memory():
    memory_data = {
        "Leslie": leslie.episodic_memory,
        "Ashley": ashley.episodic_memory
    }
    with open(memory_file, "w") as f:
        json.dump(memory_data, f, indent=4)

# Function to load agent memory
def load_memory():
    if os.path.exists(memory_file):
        with open(memory_file, "r") as f:
            memory_data = json.load(f)
        leslie.episodic_memory = memory_data.get("Leslie", [])
        ashley.episodic_memory = memory_data.get("Ashley", [])

# Execute stock chart analysis
def analyze_stock_chart(symbol):
    print("\n=== Capturing TradingView Chart ===")
    chart_path = capture_tradingview_chart(symbol)
    if not chart_path:
        return "Chart capture failed."
    
    print("\n=== Agents Reading Image ===")
    # Instead of reading the document directly, we'll have the agents think about the chart
    leslie.think(f"Analyzing the chart for {symbol} at {chart_path}")
    ashley.think(f"Examining the technical patterns in {symbol}'s chart at {chart_path}")
    
    print("\n=== AI Analysis ===")
    ai_analysis = analyze_chart_with_gpt4o(chart_path)
    print(f"AI Analysis: {ai_analysis}")
    
    print("\n=== Agents Responding ===")
    leslie_response = get_gpt_response(f"Leslie, provide an expert opinion on the AI's analysis: {ai_analysis}")
    ashley_response = get_gpt_response(f"Ashley, respond to Leslie's thoughts: {leslie_response}")
    
    # Have agents process the responses through thinking
    leslie.think(leslie_response)
    ashley.think(ashley_response)
    
    # Run a few rounds of interaction
    environment.run(4)
    
    save_memory()
    return ai_analysis

# Load memory before running
environment.run(2)
load_memory()

if __name__ == "__main__":
    # Run for a sample stock
    analyze_stock_chart("SPY")


In [ ]:
TinyPerson.clear_agents()

In [ ]:
environment.clear_environments()

In [ ]:
import os
import time
import json
from dotenv import load_dotenv
from openai import OpenAI
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from imagekitio import ImageKit
from PIL import Image
import tinytroupe
from tinytroupe.agent import TinyPerson
from tinytroupe.environment import TinyWorld

# Load environment variables
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
IMAGEKIT_PRIVATE_KEY = os.getenv("IMAGEKIT_PRIVATE_KEY")
IMAGEKIT_PUBLIC_KEY = os.getenv("IMAGEKIT_PUBLIC_KEY")
IMAGEKIT_URL_ENDPOINT = os.getenv("IMAGEKIT_URL_ENDPOINT")
client = OpenAI(api_key=OPENAI_API_KEY)

# Initialize ImageKit
imagekit = ImageKit(
    private_key=IMAGEKIT_PRIVATE_KEY,
    public_key=IMAGEKIT_PUBLIC_KEY,
    url_endpoint=IMAGEKIT_URL_ENDPOINT
)

# Define directories
screenshot_dir = "screenshots"
os.makedirs(screenshot_dir, exist_ok=True)
memory_file = "agent_memory.json"

# Initialize agents
leslie = TinyPerson("Leslie")
leslie.define("age", 75)
leslie.define("occupation", "Baker, Mechanic, Accountant")
leslie.define("personality", {"traits": [
    "Patient and analytical.",
    "Enjoys explaining solutions to problems with numbers.",
    "Friendly and a good active listener."
]})

ashley = TinyPerson("Ashley")
ashley.define("age", 21)
ashley.define("occupation", "Model, Chemist, Gymnast")
ashley.define("personality", {"traits": [
    "Curious and analytical.",
    "Loves cooking healthy food.",
    "Very playful and energetic."
]})

# Create world
environment = TinyWorld("Trading Room", [leslie, ashley])
environment.make_everyone_accessible()

# Function to query OpenAI's GPT-4o
def get_gpt_response(prompt):
        completions = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a stock market expert analyzing candlestick charts."},
                {"role": "user", "content": [
                    {"type": "text", "text": "Analyze the stock chart in this image and provide insights on trends, volatility, and possible price action. Limit your response to 650 tokens and indicate the perceived sentiment."},
                    {"type": "image_url", "image_url": {"url": image_url}}
                ]}
            ]
        )
        return completions

# Function to capture TradingView chart
def capture_tradingview_chart(symbol):
    try:
        options = Options()
        options.add_argument("--window-size=1920x1080")
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(f"https://www.tradingview.com/chart/?symbol={symbol}")
        time.sleep(5)
        screenshot_path = os.path.join(screenshot_dir, f"{symbol}_chart.png")
        driver.save_screenshot(screenshot_path)
        driver.quit()
        return screenshot_path
    except Exception as e:
        print(f"Error capturing chart: {e}")
        return None

# Function to upload image to ImageKit
def upload_image_to_imagekit(image_path):
    try:
        if not os.path.exists(image_path):
            return "Error: Image file not found."
        
        print(f"Uploading {image_path} to ImageKit...")
        with open(image_path, "rb") as img_file:
            upload = imagekit.upload(
                file=img_file,
                file_name=os.path.basename(image_path)
            )
        return upload.response_metadata.raw['url']
    except Exception as e:
        print(f"Error uploading image: {e}")
        return None

# Function to analyze chart using GPT-4o Vision
def analyze_chart_with_gpt4o(image_url):
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "Analyze the stock chart image and provide insights."},
                {"role": "user", "content": [
                    {"type": "text", "text": "Describe trends, volatility, and possible price action."},
                    {"type": "image_url", "image_url": {"url": image_url}}
                ]}
            ]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error analyzing chart: {e}"

# Execute stock chart analysis
def analyze_stock_chart(symbol):
    print("\n=== Capturing TradingView Chart ===")
    chart_path = capture_tradingview_chart(symbol)
    if not chart_path:
        return "Chart capture failed."
    
    print("\n=== Uploading Image to ImageKit ===")
    image_url = upload_image_to_imagekit(chart_path)
    if not image_url or "Error" in image_url:
        return "Image upload failed."
    
    print("\n=== AI Analysis ===")
    ai_analysis = analyze_chart_with_gpt4o(image_url)
    print(f"AI Analysis: {ai_analysis}")
    
    print("\n=== Agents Responding ===")
    leslie_response = get_gpt_response(f"Leslie, provide an expert opinion on the AI's analysis: {ai_analysis}")
    ashley_response = get_gpt_response(f"Ashley, respond to Leslie's thoughts: {leslie_response}")
    
    leslie.listen(leslie_response)
    ashley.listen(ashley_response)
    environment.run(4)
    
    return ai_analysis

# Run for a sample stock
analyze_stock_chart("SPY")
